In [49]:
import pandas as pd
df = pd.read_csv('hbp3.tsv', delimiter='\t')

In [50]:
df.head()

,DATETIME(JST),DATENAME(JST),HOURS(JST),DATENAME,DATETIME,HOURS,Table Name,building,floor,sensorId,stall,status,unixtimestamp
0,2017/12/20 13:13,Wednesday,13,Wednesday,2017/12/20 4:13,4,1513743187638.json,91,4,910401,1,close,1.510000e+12
1,2017/12/20 13:16,Wednesday,13,Wednesday,2017/12/20 4:16,4,1513743383454.json,91,4,910403,3,close,1.510000e+12
2,2017/12/20 13:21,Wednesday,13,Wednesday,2017/12/20 4:21,4,1513743674523.json,91,4,910402,2,close,1.510000e+12
3,2017/12/20 13:23,Wednesday,13,Wednesday,2017/12/20 4:23,4,1513743794829.json,91,4,910403,3,open,1.510000e+12
4,2017/12/20 13:23,Wednesday,13,Wednesday,2017/12/20 4:23,4,1513743835781.json,91,4,910403,3,close,1.510000e+12


In [51]:
unixt = df["Table Name"].str.extract('([0-9]+)')

C:\Users\shoichi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [52]:
dat = df.loc[:,["DATETIME(JST)","DATENAME(JST)","HOURS(JST)","sensorId","stall","status"]]

In [53]:
dat = pd.concat([dat, unixt], axis=1)
#カラムネームの変更を行う
columns={
"Table Name":"unixtimestamp"
}
#再度読み込みを行う
dat.rename(columns=columns, inplace=True)
dat.head()

,DATETIME(JST),DATENAME(JST),HOURS(JST),sensorId,stall,status,unixtimestamp
0,2017/12/20 13:13,Wednesday,13,910401,1,close,1513743187638
1,2017/12/20 13:16,Wednesday,13,910403,3,close,1513743383454
2,2017/12/20 13:21,Wednesday,13,910402,2,close,1513743674523
3,2017/12/20 13:23,Wednesday,13,910403,3,open,1513743794829
4,2017/12/20 13:23,Wednesday,13,910403,3,close,1513743835781


In [60]:
dat1 = dat[dat.stall==1]
dat2 = dat[dat.stall==2]
dat3 = dat[dat.stall==3]

In [68]:
for i in range(len(dat1)):
    try:
        if dat1.iloc[i]["status"] == "open" and dat1.iloc[i+1]["status"] == "close":
#        print(str(i))
            cls = dat1.iloc[[i]]
            opn = dat1.iloc[[i+1]]
            d = pd.concat([cls, opn])
            d.to_csv("preset/"+str(i)+".csv", index=False)
    except:
        print('indexer is out-of-bounds')

indexer is out-of-bounds


In [71]:
import glob
path = "preset/" # use your path
all_files = glob.glob(path + "*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in all_files:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
frame = pd.concat(list_)
stall1 = frame.sort_values(by=['unixtimestamp'])
stall1.to_csv('stall1.csv', index=None)

In [95]:
cls1 = stall1[stall1.status == "close"].reset_index(drop=True)
op1 = stall1[stall1.status == "open"].reset_index(drop=True)

In [96]:
cls1.head()

,DATETIME(JST),DATENAME(JST),HOURS(JST),sensorId,stall,status,unixtimestamp
0,2017/12/20 13:45,Wednesday,13,910401,1,close,1513745121391
1,2017/12/20 13:53,Wednesday,13,910401,1,close,1513745607821
2,2017/12/20 14:03,Wednesday,14,910401,1,close,1513746208398
3,2017/12/20 14:08,Wednesday,14,910401,1,close,1513746485029
4,2017/12/20 14:11,Wednesday,14,910401,1,close,1513746669042


In [97]:
#カラムネームの変更を行う
columns={
    "DATETIME(JST)":"close_time",
    "DATENAME(JST)":"DATENAME",
    "HOURS(JST)":"hour_jst"
}
#再度読み込みを行う
cls1.rename(columns=columns, inplace=True)

In [98]:
#カラムネームの変更を行う
columns={
    "DATETIME(JST)":"open_time",
    "DATENAME(JST)":"DATENAME",
    "HOURS(JST)":"hour_jst"
}
#再度読み込みを行う
op1.rename(columns=columns, inplace=True)

In [99]:
op1["open_time"].head()

0    2017/12/20 13:32
1    2017/12/20 13:50
2    2017/12/20 13:59
3    2017/12/20 14:07
4    2017/12/20 14:10
Name: open_time, dtype: object

In [100]:
cls1["close_time"].head()

0    2017/12/20 13:45
1    2017/12/20 13:53
2    2017/12/20 14:03
3    2017/12/20 14:08
4    2017/12/20 14:11
Name: close_time, dtype: object

In [101]:
len(op1)

1198

In [102]:
len(cls1)

1198

In [104]:
f1 = pd.concat([op1["open_time"], cls1], axis=1)

In [105]:
f1.head()

,open_time,close_time,DATENAME,hour_jst,sensorId,stall,status,unixtimestamp
0,2017/12/20 13:32,2017/12/20 13:45,Wednesday,13,910401,1,close,1513745121391
1,2017/12/20 13:50,2017/12/20 13:53,Wednesday,13,910401,1,close,1513745607821
2,2017/12/20 13:59,2017/12/20 14:03,Wednesday,14,910401,1,close,1513746208398
3,2017/12/20 14:07,2017/12/20 14:08,Wednesday,14,910401,1,close,1513746485029
4,2017/12/20 14:10,2017/12/20 14:11,Wednesday,14,910401,1,close,1513746669042


In [106]:
f1.to_csv('f1.csv',index=False)

In [107]:
for i in range(len(dat2)):
    try:
        if dat2.iloc[i]["status"] == "open" and dat2.iloc[i+1]["status"] == "close":
#        print(str(i))
            cls = dat2.iloc[[i]]
            opn = dat2.iloc[[i+1]]
            d = pd.concat([cls, opn])
            d.to_csv("preset2/"+str(i)+".csv", index=False)
    except:
        print('indexer is out-of-bounds')

indexer is out-of-bounds


In [108]:
import glob
path = "preset2/" # use your path
all_files = glob.glob(path + "*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in all_files:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
frame = pd.concat(list_)
stall2 = frame.sort_values(by=['unixtimestamp'])
stall2.to_csv('stall2.csv', index=None)

In [109]:
cls2 = stall2[stall2.status == "close"].reset_index(drop=True)
op2 = stall2[stall2.status == "open"].reset_index(drop=True)

In [110]:
#カラムネームの変更を行う
columns={
    "DATETIME(JST)":"close_time",
    "DATENAME(JST)":"DATENAME",
    "HOURS(JST)":"hour_jst"
}
#再度読み込みを行う
cls2.rename(columns=columns, inplace=True)

#カラムネームの変更を行う
columns={
    "DATETIME(JST)":"open_time",
    "DATENAME(JST)":"DATENAME",
    "HOURS(JST)":"hour_jst"
}
#再度読み込みを行う
op2.rename(columns=columns, inplace=True)

In [111]:
f2 = pd.concat([op2["open_time"], cls2], axis=1)
f2.to_csv('f2.csv',index=False)